<a href="https://colab.research.google.com/github/mmartigani/Pandas_con_python/blob/main/Scikit_Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df_review = pd.read_csv('IMDB Dataset.csv')
df_review

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
#lo que hacemos es reducir el dataset a 10000
df_positive = df_review[df_review['sentiment']=='positive'][:9000]
df_negative = df_review[df_review['sentiment']=='negative'][:1000]

df_review_des = pd.concat([df_positive, df_negative])
df_review_des.value_counts(['sentiment']) #cuenta los positivos y negativos 

sentiment
positive     9000
negative     1000
dtype: int64

In [ ]:
#Balanceando Dataset
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler()
df_review_bal, df_review_bal['sentiment'] = rus.fit_resample(df_review_des[['review']],
                                                          df_review_des['sentiment'])

df_review_bal.value_counts(['sentiment'])

sentiment
negative     1000
positive     1000
dtype: int64

In [ ]:
#Separando data para entrenar (train) y testear (test)
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_review_bal, test_size=0.33, random_state=42)

In [ ]:
train_x, train_y = train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']

In [ ]:
#transformamos la data text a data numerica 
#Representacion de Text (Bag of Words)
#CountVectorizer
#Tfidf

In [ ]:
#frecuencia que una palabra aparece en el texto
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
text = ["Amo escribir codigo en Python. Amo el código en Python",
        "Odio escribir codigo en Java. Odio el código en Java"]

df = pd.DataFrame({'review': ['review1', 'review2'], 'text':text})
cv = CountVectorizer()
cv_matrix = cv.fit_transform(df['text'])
df_dtm = pd.DataFrame(cv_matrix.toarray(), index=df['review'].values, columns=cv.get_feature_names())
df_dtm

,amo,codigo,código,el,en,escribir,java,odio,python
review1,2,1,1,1,2,1,0,0,2
review2,0,1,1,1,2,1,2,2,0


In [ ]:
#Tfidf (term frequency - inverse document frequency)
#le da relevancia a palabras unicas representativa que aparace muchas 
#veces en un reviews  pero no esta en los demas reviews
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
text = ["Amo escribir codigo en Python. Amo el código en Python",
        "Odio escribir codigo en Java. Odio el código en Java"]

df = pd.DataFrame({'review': ['review1', 'review2'], 'text':text})
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['text'])
df_dtm = pd.DataFrame(tfidf_matrix.toarray(), index=df['review'].values, columns=tfidf.get_feature_names())
df_dtm

,amo,codigo,código,el,en,escribir,java,odio,python
review1,0.576152,0.204969,0.204969,0.204969,0.409937,0.204969,0.000000,0.000000,0.576152
review2,0.000000,0.204969,0.204969,0.204969,0.409937,0.204969,0.576152,0.576152,0.000000


In [ ]:
#Transformar data de texto a data numerica
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
train_x_vector = tfidf.fit_transform(train_x)

test_x_vector = tfidf.transform(test_x)

In [ ]:
train_x_vector

<1340x20273 sparse matrix of type '<class 'numpy.float64'>'
	with 115312 stored elements in Compressed Sparse Row format>

In [ ]:
#Seleccion del Modelo
#ML algoritmos
#debemos saber que tipo de problema tenemos

#Aprendizaje supervisado (Supervised learning): #2 ramas regresion y clasificacion 
#Regresion (output numerico),(1,0)
#Clasificacion (output discreto) (positivo , negativo)
#____-------------------------------
#Input: Review
#Output: Sentiment (discrete)
#Aprendizaje no supervisado

In [ ]:
#Support Vector Machines (SVM)

In [ ]:
from sklearn.svm import SVC

svc = SVC(kernel='linear')
svc.fit(train_x_vector, train_y)

SVC(kernel='linear')

In [ ]:
#Testeo

In [ ]:
print(svc.predict(tfidf.transform(['A good movie']))) #buena pelicula
print(svc.predict(tfidf.transform(['An excellent movie']))) #excelente pelicula
print(svc.predict(tfidf.transform(['"I did not like this movie at all I gave this movie away"'])))# no gusto

['positive']
['positive']
['negative']


In [ ]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_x_vector, train_y)

DecisionTreeClassifier()

In [ ]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(train_x_vector.toarray(), train_y)

GaussianNB()

In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_x_vector, train_y)

LogisticRegression()

In [ ]:
#Evaluacion del Modelo
#Score (Accuracy)saber la precision
print(svc.score(test_x_vector, test_y))#modelo con mejor precision 
print(dec_tree.score(test_x_vector, test_y))
print(gnb.score(test_x_vector.toarray(), test_y))
print(lr.score(test_x_vector, test_y))

0.8424242424242424
0.7151515151515152
0.6106060606060606
0.8318181818181818


In [ ]:
#F1 Score
#F1 Score = 2(Recall Precision) / (Recall + Precision)
from sklearn.metrics import f1_score

f1_score(test_y, svc.predict(test_x_vector),
         labels=['positive', 'negative'],
         average=None)

array([0.84705882, 0.8375    ])

In [ ]:
#Reporte de Clasificacion
from sklearn.metrics import classification_report

print(classification_report(test_y, svc.predict(test_x_vector),
                      labels=['positive', 'negative']))

              precision    recall  f1-score   support

    positive       0.83      0.86      0.85       335
    negative       0.85      0.82      0.84       325

    accuracy                           0.84       660
   macro avg       0.84      0.84      0.84       660
weighted avg       0.84      0.84      0.84       660



In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, svc.predict(test_x_vector),
                 labels=['positive', 'negative'])

array([[288,  47],
       [ 57, 268]], dtype=int64)

In [ ]:
#Optimizacion del Model
#GridSearchCV
from sklearn.model_selection import GridSearchCV

paremetros = {'C':[1,4,8,16,32], 'kernel':['linear', 'rbf']}
svc = SVC()
svc_grid = GridSearchCV(svc, paremetros, cv=5)
svc_grid.fit(train_x_vector, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 4, 8, 16, 32], 'kernel': ['linear', 'rbf']})

In [ ]:
#comenatrio para saber que parametros incluyen dentro de parentesis .toarray()
#colocamos shit +tab

In [ ]:
print(svc_grid.best_estimator_)
print(svc_grid.best_params_)

SVC(C=4)
{'C': 4, 'kernel': 'rbf'}


In [ ]:
svc_grid.best_score_

0.8365671641791044